<a href="https://colab.research.google.com/github/KimJisanER/code_jjambbong/blob/main/3D_CNN_0721.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
with open(file='/content/drive/MyDrive/dataset_5_HTR/train/train_data.pickle', mode='rb') as f:
    train_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/train/train_3D_data.pickle', mode='rb') as f:
    train_3D_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/test/test_data.pickle', mode='rb') as f:
    test_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/test/test_3D_data.pickle', mode='rb') as f:
    test_3D_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/validation/val_data.pickle', mode='rb') as f:
    val_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/validation/val_3D_data.pickle', mode='rb') as f:
    val_3D_data = pickle.load(f)

In [3]:
train_data

,target_accession,chembl_id,Fingerprint,pChEMBL_value_Ki
7911,P28223,CHEMBL311614,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.16
11866,P28335,CHEMBL52592,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",8.00
6067,P41595,CHEMBL299031,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",5.90
14159,P50406,CHEMBL4075727,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.28
5208,P28221,CHEMBL519037,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",5.70
...,...,...,...,...
8251,P28223,CHEMBL3577344,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",5.44
4289,P08908,CHEMBL598561,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.62
509,P28222,CHEMBL370896,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",7.90
15539,P34969,CHEMBL3401491,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.05


In [3]:
# def convert_data(data):
#   data_array = np.array([dt for dt in data])
#   return data_array

def convert_data(data):
    data_array = np.array([dt for dt in data])
    data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
    return data_tensor

def convert_fpdata(data):
  fpdata_array = np.array([keras.utils.to_categorical(fp, num_classes=2) for fp in data])
  return fpdata_array

In [4]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tqdm import tqdm
import os

# Mixed Precision Training 활성화
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

save_dir = "/content/drive/MyDrive/dataset_5_HTR/3D_model"

@tf.function
def train_one_epoch(model, data_loader, loss_fn, optimizer):
    total_loss = 0.0
    num_batches = 0

    for inputs1, inputs2, labels in tqdm(data_loader):
        with tf.GradientTape() as tape:
            # 모델의 순방향 전파
            logits = model([inputs1, inputs2])

            # 손실 계산
            loss_value = loss_fn(labels, logits)

        # 그래디언트 계산
        grads = tape.gradient(loss_value, model.trainable_variables)

        # 옵티마이저를 사용하여 모델 가중치 업데이트
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # 훈련 결과 출력
        total_loss += loss_value
        num_batches += 1

    average_loss = total_loss / num_batches
    return average_loss

@tf.function
def evaluate(model, data_loader, loss_fn):
    total_loss = 0.0
    num_batches = 0

    for inputs1, inputs2, labels in data_loader:
        logits = model([inputs1, inputs2])
        loss_value = loss_fn(labels, logits)

        total_loss += loss_value
        num_batches += 1

    average_loss = total_loss / num_batches
    return average_loss

def preprocess_data(dataframe, dictionary):
    fingerprint = dataframe['Fingerprint'].apply(convert_data).values.tolist()
    labels = dataframe['pChEMBL_value_Ki'].astype(np.float32).values
    dictionary_values = [dictionary[i] for i in dataframe.index]
    return fingerprint, dictionary_values, labels


# def get_dataset(dataframe, dictionary, batch_size=8, shuffle=True, num_parallel_calls=tf.data.AUTOTUNE):
#     indexes = dataframe.index.tolist()
#     num_samples = len(indexes)

#     def generator():
#         while True:
#             if shuffle:
#                 np.random.shuffle(indexes)
#             for i in range(0, num_samples, batch_size):
#                 batch_indexes = indexes[i:i + batch_size]
#                 batch_data_frame_rows = [dataframe['Fingerprint'].loc[i] for i in batch_indexes]
#                 batch_labels = [dataframe['pChEMBL_value_Ki'].loc[i].astype(np.float32) for i in batch_indexes]
#                 batch_dictionary_rows = [dictionary[i] for i in batch_indexes]

#                 batch_data_frame_rows = [convert_data(row) for row in batch_data_frame_rows]

#                 yield batch_dictionary_rows, batch_data_frame_rows, batch_labels

#     output_shapes = ([batch_size, 128, 128, 128, 24], [batch_size, 2048, 2], [batch_size, 1])
#     dataset = tf.data.Dataset.from_generator(generator, (tf.float32, tf.float32, tf.float32), output_shapes=output_shapes)
#     dataset = dataset.map(lambda x, y, z: (tf.convert_to_tensor(x), tf.convert_to_tensor(y), tf.convert_to_tensor(z)), num_parallel_calls=num_parallel_calls)
#     return dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


def get_dataset(dataframe, dictionary, batch_size=8, shuffle=True, num_workers=None):
    indexes = dataframe.index.tolist()
    num_samples = len(indexes)

    def generator():
        while True:
            if shuffle:
                np.random.shuffle(indexes)
            for i in range(0, num_samples, batch_size):
                batch_indexes = indexes[i:i + batch_size]
                batch_data_frame_rows = [dataframe['Fingerprint'].loc[i] for i in batch_indexes]
                batch_labels = [dataframe['pChEMBL_value_Ki'].loc[i].astype(np.float32) for i in batch_indexes]
                batch_dictionary_rows = [dictionary[i] for i in batch_indexes]

                batch_data_frame_rows = [convert_data(row) for row in batch_data_frame_rows]

                yield batch_dictionary_rows, batch_data_frame_rows, batch_labels

    output_shapes = ([batch_size, 128, 128, 128, 24], [batch_size, 2048, 2], [batch_size, 1])
    dataset = tf.data.Dataset.from_generator(generator, (tf.float32, tf.float32, tf.float32), output_shapes=output_shapes)

    # Use tf.data.experimental.AUTOTUNE for num_workers if not provided
    num_workers = num_workers if num_workers is not None else tf.data.experimental.AUTOTUNE

    dataset = dataset.map(lambda x, y, z: (tf.convert_to_tensor(x), tf.convert_to_tensor(y), tf.convert_to_tensor(z)), num_parallel_calls=num_workers)
    return dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


def train(model, train_data_loader, valid_data_loader, loss_fn, optimizer, num_epochs):
    train_losses = []
    valid_losses = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # 훈련
        train_loss = train_one_epoch(model, train_data_loader, loss_fn, optimizer)
        train_losses.append(train_loss)
        print(f"Train Loss: {train_loss}")

        # 검증
        valid_loss = evaluate(model, valid_data_loader, loss_fn)
        valid_losses.append(valid_loss)
        print(f"Validation Loss: {valid_loss}")

        model.save(os.path.join(save_dir, f"model_epoch_{epoch + 1}_batch.h5"))

    return train_losses, valid_losses


In [6]:
# class CustomDataLoader(tf.keras.utils.Sequence):
#     def __init__(self, dataframe, dictionary):
#         self.dataframe = dataframe
#         self.dictionary = dictionary
#         self.indexes = dataframe.index.tolist()  # 데이터 프레임의 인덱스 사용

#     def __len__(self):
#         return len(self.indexes)

#     def __getitem__(self, index):
#         index_value = self.indexes[index]
#         data_frame_row = self.dataframe['Fingerprint'].loc[index_value]
#         label = self.dataframe['pChEMBL_value_Ki'].loc[index_value].astype(np.float32)
#         dictionary_row = self.dictionary[index_value]

#         data_frame_row = convert_data(data_frame_row)

#         # 텐서로 변환 (예시: 텐서플로우의 from_tensor_slices 메서드 사용)
#         data_frame_tensor = tf.convert_to_tensor(data_frame_row)
#         dictionary_tensor = tf.convert_to_tensor(dictionary_row)
#         labels_tensor = tf.convert_to_tensor(label)

#         data_frame_tensor = tf.expand_dims(tf.convert_to_tensor(data_frame_row), axis=0)
#         dictionary_tensor = tf.expand_dims(tf.convert_to_tensor(dictionary_row), axis=0)

#         return  dictionary_tensor, data_frame_tensor, labels_tensor


In [7]:
# from tensorflow.keras.layers import LeakyReLU
# from tensorflow.keras.layers import ReLU

# # 이미지 입력 경로
# image_input_shape = (128, 128, 128, 24)
# image_input = keras.Input(shape=image_input_shape)
# x1_5x5 = layers.Conv3D(64, (5, 5, 5), padding='same')(image_input)
# x1_5x5 = LeakyReLU(alpha=0.2)(x1_5x5)
# x1_5x5 = layers.Conv3D(64, (5, 5, 5), padding='same')(x1_5x5)
# x1_5x5 = LeakyReLU(alpha=0.2)(x1_5x5)

# x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
# x1_5x5 = layers.Conv3D(128, (5, 5, 5), padding='same')(x1_5x5)
# x1_5x5 = LeakyReLU(alpha=0.2)(x1_5x5)
# x1_5x5 = layers.Conv3D(128, (5, 5, 5), padding='same')(x1_5x5)
# x1_5x5 = LeakyReLU(alpha=0.2)(x1_5x5)

# x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
# x1_5x5 = layers.Flatten()(x1_5x5)

# x1 = x1_5x5
# x1 = layers.Dense(256)(x1)
# x1 = ReLU()(x1)
# x1 = layers.BatchNormalization()(x1)
# x1 = layers.Dense(128)(x1)
# x1 = ReLU()(x1)
# x1 = layers.BatchNormalization()(x1)

# # 비트 입력 경로
# bit_input_shape = shape=(2048, 2)
# bit_input = keras.Input(shape=bit_input_shape)
# x2 = layers.Flatten()(bit_input)

# # 경로 병합
# merged = layers.concatenate([x1, x2])
# merged = layers.Dense(512)(merged)
# merged = ReLU()(merged)
# merged = layers.BatchNormalization()(merged)
# merged = layers.Dropout(0.1)(merged)
# merged = layers.Dense(256)(merged)
# merged = ReLU()(merged)
# merged = layers.BatchNormalization()(merged)
# merged = layers.Dropout(0.1)(merged)
# merged = layers.Dense(128)(merged)
# merged = ReLU()(merged)
# merged = layers.BatchNormalization()(merged)
# merged = layers.Dropout(0.1)(merged)
# merged = layers.Dense(64)(merged)
# merged = ReLU()(merged)
# merged = layers.BatchNormalization()(merged)


# # 회귀 모델 출력 레이어 추가
# outputs = layers.Dense(1)(merged)

# # 모델 생성
# model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

# # 모델 컴파일
# # model.compile(optimizer='adam', loss='mse')
# loss_fn = tf.keras.losses.MeanSquaredError()
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

# # 모델 요약
# model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 24)]                                                          
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  192064      ['input_1[0][0]']                
                                8, 64)                                                            
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 128, 128, 12  0           ['conv3d[0][0]']                 
                                8, 64)                                                        

In [6]:
# 이미지 입력 경로
image_input_shape = (128, 128, 128, 24)
image_input = keras.Input(shape=image_input_shape)
x1_5x5 = layers.Conv3D(64, (5, 5, 5), padding='same', activation='relu')(image_input)
x1_5x5 = layers.Conv3D(64, (5, 5, 5), padding='same', activation='relu')(x1_5x5)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Conv3D(128, (5, 5, 5), padding='same', activation='relu')(x1_5x5)
x1_5x5 = layers.Conv3D(128, (5, 5, 5), padding='same', activation='relu')(x1_5x5)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Conv3D(256, (5, 5, 5), padding='same', activation='relu')(x1_5x5)
x1_5x5 = layers.Conv3D(256, (5, 5, 5), padding='same', activation='relu')(x1_5x5)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Flatten()(x1_5x5)

x1 = x1_5x5
x1 = layers.Dense(256, activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dropout(0.1)(x1)
x1 = layers.Dense(128, activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)

# 비트 입력 경로
bit_input_shape = shape=(2048, 2)
bit_input = keras.Input(shape=bit_input_shape)
x2 = layers.Flatten()(bit_input)

# 경로 병합
merged = layers.concatenate([x1, x2])
merged = layers.Dense(512, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)
merged = layers.Dropout(0.1)(merged)
merged = layers.Dense(256, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)
merged = layers.Dropout(0.1)(merged)
merged = layers.Dense(128, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)

# 회귀 모델 출력 레이어 추가
outputs = layers.Dense(1)(merged)

# 모델 생성
model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

# 모델 컴파일
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss=loss_fn)



# 모델 요약
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 24)]                                                          
                                                                                                  
 conv3d_6 (Conv3D)              (None, 128, 128, 12  192064      ['input_3[0][0]']                
                                8, 64)                                                            
                                                                                                  
 conv3d_7 (Conv3D)              (None, 128, 128, 12  512064      ['conv3d_6[0][0]']               
                                8, 64)                                                      

In [7]:
# Create train and validation datasets using tf.data.Dataset
train_data_loader = get_dataset(train_data.iloc[:, 2:], train_3D_data, batch_size=8, shuffle=False, num_workers=8)
val_data_loader = get_dataset(val_data.iloc[:, 2:], val_3D_data, batch_size=8, shuffle=False, num_workers=8)


# Train the model
num_epochs = 3
# train_losses, valid_losses = train(model, train_data_loader, val_data_loader, loss_fn, optimizer, num_epochs)


# Train the model using model.fit
num_epochs = 3
history = model.fit(
    train_data_loader,
    epochs=num_epochs,
    validation_data=val_data_loader,
    verbose=1
)

# Get the training and validation losses from history
train_losses = history.history['loss']
valid_losses = history.history['val_loss']

Epoch 1/3


ValueError: ignored

In [11]:
# import tensorflow as tf
# from tensorflow.keras import mixed_precision
# from tqdm import tqdm

# # Mixed Precision Training 활성화
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)

# save_dir = "/content/drive/MyDrive/dataset_5_HTR/3D_model"


# def train_one_epoch(model, data_loader, loss_fn, optimizer):
#     total_loss = 0.0
#     num_batches = 0

#     for inputs1, inputs2, labels in tqdm(data_loader):
#         with tf.GradientTape() as tape:
#             # 모델의 순방향 전파
#             logits = model([inputs1, inputs2])

#             # 손실 계산
#             loss_value = loss_fn(labels, logits)

#         # 그래디언트 계산
#         grads = tape.gradient(loss_value, model.trainable_variables)

#         # 옵티마이저를 사용하여 모델 가중치 업데이트
#         optimizer.apply_gradients(zip(grads, model.trainable_variables))

#         # 훈련 결과 출력
#         total_loss += loss_value
#         num_batches += 1
#         # print(f'Loss:{loss_value.numpy()} experimental: {labels} predicted: {logits}' )

#     average_loss = total_loss / num_batches
#     return average_loss

# def evaluate(model, data_loader, loss_fn):
#     total_loss = 0.0
#     num_batches = 0

#     for inputs1, inputs2, labels in data_loader:
#         logits = model([inputs1, inputs2])
#         print(logits)
#         loss_value = loss_fn(labels, logits)

#         total_loss += loss_value.numpy()
#         num_batches += 1

#     average_loss = total_loss / num_batches
#     return average_loss

# def train(model, train_data_loader, valid_data_loader, loss_fn, optimizer, num_epochs):
#     train_losses = []
#     valid_losses = []

#     for epoch in range(num_epochs):
#         print(f"Epoch {epoch + 1}/{num_epochs}")

#         # 훈련
#         train_loss = train_one_epoch(model, train_data_loader, loss_fn, optimizer)
#         train_losses.append(train_loss)
#         print(f"Train Loss: {train_loss}")

#         # 검증
#         valid_loss = evaluate(model, valid_data_loader, loss_fn)
#         valid_losses.append(valid_loss)
#         print(f"Validation Loss: {valid_loss}")

#         model.save(os.path.join(save_dir, f"model_epoch_{epoch + 1}_3x3_4layer.h5"))

#     return train_losses, valid_losses

In [12]:
# Create train and validation datasets using tf.data.Dataset
train_data_loader = get_dataset(train_data.iloc[:, 2:], train_3D_data, batch_size=8, shuffle=False, num_workers=8)
val_data_loader = get_dataset(val_data.iloc[:, 2:], val_3D_data, batch_size=8, shuffle=False, num_workers=8)


# Train the model
num_epochs = 3
train_losses, valid_losses = train(model, train_data_loader, val_data_loader, loss_fn, optimizer, num_epochs)

Epoch 1/3


32it [01:01,  1.93s/it]


KeyboardInterrupt: ignored

In [9]:
batch_size = 8
num_threads = 4
prefetch_buffer = 1

batch_size = 8
num_threads = 4
prefetch_buffer = 1

train_data_loader = CustomDataLoader(train_data.iloc[: , 2:], train_3D_data, batch_size=batch_size, num_threads=num_threads, prefetch_buffer=prefetch_buffer)
val_data_loader = CustomDataLoader(val_data.iloc[: , 2:], val_3D_data, batch_size=batch_size, num_threads=num_threads, prefetch_buffer=prefetch_buffer)

# 모델 훈련
num_epochs = 3
train_losses, valid_losses = train(model, train_data_loader, val_data_loader, loss_fn, optimizer, num_epochs)

Epoch 1/3


  0%|          | 0/1658 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
def evaluate(model, data_loader, loss_fn):
    total_loss = 0.0
    num_batches = 0

    for inputs1, inputs2 in data_loader:
        logits = model([inputs1, inputs2])
        print(logits)
        labels = inputs2[:, -1]
        loss_value = loss_fn(labels, logits)

        total_loss += loss_value.numpy()
        num_batches += 1

    average_loss = total_loss / num_batches
    return average_loss

In [ ]:
test_data_loader = CustomDataLoader(test_data.iloc[: , 2:], test_3D_data)

In [ ]:
evaluate(model, test_data_loader, loss_fn)

tf.Tensor([[6.8239546]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.9718256]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.7061443]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.137581]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.540615]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.996312]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.167155]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.953685]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.334463]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.7186604]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.7656183]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.9808693]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.509247]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.152228]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.5831766]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.2787914]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.3950186]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.837284]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.2832727]], shape=(1, 1)

0.9103905382871924

In [ ]:
model.predict()